In [223]:
import numpy as np
# Preparation
zero = np.array([1,0])
one = np.array([0,1])

one3 = np.kron(np.kron(one,one),one)
zero3 = np.kron(np.kron(zero,zero),zero)

zeroPlusone = 1/np.sqrt(2)*(zero3 + one3)
zeroMinusone = 1/np.sqrt(2)*(zero3 - one3)

# Logic 0 and 1
zeroL = np.kron(np.kron(zeroPlusone,zeroPlusone),zeroPlusone)
oneL  = np.kron(np.kron(zeroMinusone,zeroMinusone),zeroMinusone)

# No errors state
a = 0.5
b = 0.1*1j
Psi = a*zeroL + b*oneL
Psi = Psi/np.sqrt(np.vdot(Psi,Psi))

# Pauli matrices
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
E = np.eye(2)

# General Tensor product of pauli matrices 
def sigma(vec_S,vec_i, N = 9):
    # Operator S_0 x S_1 x ... x S_N
    # vec_i contains the ORDERED!!! positions of the operators S_i in the tensor product
    # indexes not inside vec_i are considered as Identity operator
    
    mat = 1
    k = 0
    vec_i = np.concatenate([vec_i, np.array([-1])])     # So you don't get errors when trying to fetch vec_i[k] and k = len
    for j in np.arange(0,N):
        if j == vec_i[k]:
            mat = np.kron(mat, vec_S[k])
            k = k + 1
        else:
            mat = np.kron(mat,np.eye(2))
    return mat

#Test
#Z0Z1 = sigma(np.array([X]),np.array([0]),2)
#print(Z0Z1)

In [224]:
# Define stabilizers
Z0Z1 = sigma(np.array([Z,Z]),np.array([0,1]))
Z1Z2 = sigma(np.array([Z,Z]),np.array([1,2]))
Z3Z4 = sigma(np.array([Z,Z]),np.array([3,4]))
Z4Z5 = sigma(np.array([Z,Z]),np.array([4,5]))
Z6Z7 = sigma(np.array([Z,Z]),np.array([6,7]))
Z7Z8 = sigma(np.array([Z,Z]),np.array([7,8]))
X0X1X2X3X4X5 = sigma(np.array([X,X,X,X,X,X]),np.array([0,1,2,3,4,5]))
X3X4X5X6X7X8 = sigma(np.array([X,X,X,X,X,X]),np.array([3,4,5,6,7,8]))

In [225]:
# Find eigenvectors of the stabilizers (eigenvalues are all +-1)
# and decompose the state Psi in the basis of its eigenvectors

def measure(O, Psi):
    eig_vec, phi_vec = np.linalg.eig(O)
    N = np.size(phi_vec[0])
    #print(N)
    c_vec = np.zeros(N, dtype = "complex_")
    P_vec = np.zeros(N)
    for n in np.arange(0, N):
        #print(np.vdot(phi_vec[n],phi_vec[n]))
        c_vec[n] = np.vdot(phi_vec[n], Psi)
        P_vec[n] = np.abs(c_vec[n])**2
    
    #Psi_new = 0
    #for n in np.arange(0,N):
    #    Psi_new = Psi_new + c_vec[n]*phi_vec[n]
    #return [1, Psi_new]

    #error = 1 - np.sum(P_vec)
    #print("Roundoff error: ", round(100*error,3), "%")
    #P_vec = P_vec/np.sum(P_vec) # "Adjust" roundoff errors

    # Collapse the wave function
    P0 = 0
    P1 = 0
    for n in np.arange(0,N):
        if eig_vec[n] == 1:
            P0 = P0 + P_vec[n]
        else:
            P1 = P1 + P_vec[n]
    # NOW you collapse
    if (np.random.rand() < P0):
        # Collapse in Autospace of +1
        measure = 1
    else:
        measure = -1
    # New wave-function
    Psi_new = 0
    for n in np.arange(0,N):
        if eig_vec[n] == measure:
            Psi_new = Psi_new + c_vec[n]*phi_vec[n]
    Psi_new = Psi_new/np.sqrt(np.vdot(Psi_new,Psi_new))

    return [measure, Psi_new]

In [226]:
measure(X0X1X2X3X4X5,Psi)[1]
#print(np.dot(X0X1X2X3X4X5, Psi)-Psi)


array([0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0. 

In [227]:
# Define operators that implement errors
def sX(i, N = 9):
    return sigma(np.array([X]), np.array([i]))
def sY(i, N = 9):
    return sigma(np.array([X]), [i])
def sZ(i, N = 9):
    return sigma(np.array([X]), [i])

# Introduce continuos errors
alpha = 1
beta1 = 0
beta2 = 1
beta3 = 1
Psi_corr = np.dot((alpha*np.eye(2**9) + beta1*sX(5) + beta2*sY(5) + beta3*sZ(5)), Psi)
np.sum(Psi_corr - Psi)

(5.547001962252291+0j)

In [228]:
# Measiure all the stabilizers
measures = np.zeros(8)
[measures[0], Psi_corr] = measure(Z0Z1,Psi_corr)
[measures[1], Psi_corr] = measure(Z1Z2,Psi_corr)
[measures[2], Psi_corr] = measure(Z3Z4,Psi_corr)
[measures[3], Psi_corr] = measure(Z4Z5,Psi_corr)
[measures[4], Psi_corr] = measure(Z6Z7,Psi_corr)
[measures[5], Psi_corr] = measure(Z7Z8,Psi_corr)
[measures[6], Psi_corr] = measure(X0X1X2X3X4X5,Psi_corr)
[measures[7], Psi_corr] = measure(X3X4X5X6X7X8,Psi_corr)

In [229]:
measures

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1., -1.])

In [230]:
# Correct errors
Psi_new = Psi_corr
if measures[0] == 1:
    if measures[1] == 1:
        Psi_new = Psi_new
    else:
        Psi_new = np.dot(sX(2), Psi_new)
else:
    if measures[1] == 1:
        Psi_new = np.dot(sX(0), Psi_new)
    else:
        Psi_new = np.dot(sX(1), Psi_new)

if measures[2] == 1:
    if measures[3] == 1:
        Psi_new = Psi_new
    else:
        Psi_new = np.dot(sX(5), Psi_new)
else:
    if measures[3] == 1:
        Psi_new = np.dot(sX(3), Psi_new)
    else:
        Psi_new = np.dot(sX(4), Psi_new)

if measures[4] == 1:
    if measures[5] == 1:
        Psi_new = Psi_new
    else:
        Psi_new = np.dot(sX(8), Psi_new)
else:
    if measures[5] == 1:
        Psi_new = np.dot(sX(6), Psi_new)
    else:
        Psi_new = np.dot(sX(7), Psi_new)


if measures[6] == 1:
    if measures[7] == 1:
        Psi_new = Psi_new
    else:
        Psi_new = np.dot(sZ(6), Psi_new)
else:
    if measures[7] == 1:
        Psi_new = np.dot(sZ(0), Psi_new)
    else:
        Psi_new = np.dot(sZ(3), Psi_new)

# Problema riscontrato
Lo stato senza errori $\ket{\Psi}$ è autostato di $X_0X_1X_2X_3X_4X_5$ con autovalore $+1$. Lo vediamo così:

In [232]:
print(np.dot(X0X1X2X3X4X5, Psi)-Psi)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.

Tuttavia, se esprimo lo stato $\ket{\Psi}$ nella base degli autovettori di $X_0X_1X_2X_3X_4X_5$, trovo che le probabilità di misurare +1 e -1 sono identiche!

In [234]:
eig_vec, phi_vec = np.linalg.eig(X0X1X2X3X4X5)
N = np.size(phi_vec[0])

c_vec = np.zeros(N, dtype = "complex_")
P_vec = np.zeros(N)
for n in np.arange(0, N):
    c_vec[n] = np.vdot(phi_vec[n], Psi)
    P_vec[n] = np.abs(c_vec[n])**2

P0 = 0
P1 = 0
for n in np.arange(0,N):
    if eig_vec[n] == 1:
        P0 = P0 + P_vec[n]
    else:
        P1 = P1 + P_vec[n]

print("Coefficients:", c_vec)
print("Prob of measuring +1: ", P0)
print("Prob of measuring -1: ", P1)

Coefficients: [ 0.24514517+0.04902903j -0.24514517-0.04902903j  0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.24514517-0.04902903j -0.24514517+0.04902903j  0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.       

Verifichiamo che questi coefficienti forniscono lo stato $\ket{\Psi}$

In [235]:
Psi_recostr = 0
for n in np.arange(0,N):
    Psi_recostr = Psi_recostr + c_vec[n]*phi_vec[n]

print(Psi-Psi_recostr)

[5.55111512e-17+1.38777878e-17j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 5.55111512e-17-1.38777878e-17j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
 0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.000000